In [60]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as pygen  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)




USERNAME = 'pleasedshibe1' #TODO: Get from front end
header = ['username', 'name', 'description', 'number of tweets', 'followers', 'verified', 'tweets', 'links', 'hashtags', 'interacting users']
df1 = pd.read_csv(f"../{USERNAME}.csv", header=None, names=header, index_col=False) #TODO: Get column directly from query


def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i


df1 = df1[df1['tweets'].notna()]
df1.reset_index(drop=True, inplace=True)


#1 Text similarity
df1 = df1[df1['tweets'].notna()]
# df1 = df1[df1['interacting users'].notna()]
df1.reset_index(drop=True, inplace=True)





In [61]:
# Load the regular expression library
import re
# Remove punctuation
df1['tweets'] = df1['tweets'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df1['tweets'] = df1['tweets'].map(lambda x: x.lower())
# Print out the first rows of papers
df1['tweets'].head()

0    the issue i have with improving or being notic...
1    @procygfx @harikama_ @zetexkindasucks @mindcap...
2    i dunno why but i guess this name better than ...
3                    @shquaresmp :0 shibe that is rude
4    @blazeshardcat tabaxi + triton = catfish @kath...
Name: tweets, dtype: object

In [62]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df1.tweets.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['the', 'issue', 'have', 'with', 'improving', 'or', 'being', 'noticed', 'is', 'imo', 'im', 'good', 'defensive', 'team', 'player', 'but', 'cant', 'for', 'example', 'carry', 'team', 'so', 'my', 'playstyle', 'relies', 'on', 'teammates', 'so', 'not', 'many']


In [63]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [64]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'get', 'lol', 'let', 'get'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [65]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized = [[word for word in doc if len(word) > 2] for doc in data_lemmatized]


[['issue', 'improve', 'notice', 'good', 'defensive', 'team', 'player', 'example', 'carry', 'team', 'playstyle', 'rely', 'teammate', 'many', 'people', 'want', 'pick', 'level', 'feel', 'tip', 'improve', 'paralakz', 'love', 'internet', 'way', 'good', 'enough', 'flavour', 'sparkling', 'water', 'good', 'normal', 'awful', 'tbh', 'motivation', 'basically', 'back', 'going', 'grind', 'jkeejay', 'buy', 'yesterday', 'think', 'regular', 'water', 'remember', 'awful', 'exist', 'people', 'actually', 'buy', 'sparkling', 'lmfao', 'wish', 'nice', 'people', 'many', 'people', 'take', 'advantage', 'maisieelmao', 'thank', 'maisbeast', 'good', 'goal', 'usually', 'good', 'goal', 'enjoy', 'insertgender', 'let', 'max', 'good', 'sideswipe', 'less', 'mechanic', 'actual', 'game', 'sorry', 'cr_ms', 'pog', 'get', 'pink', 'title', 'season', 'going', 'grind', 'switchrl', 'sound', 'awesome', 'unfortunately', 'sure', 'fall', 'bubble', 'player', 'still', 'lft', 'next', 'regional', 'rezear', 'freakiirl', 'alemzcrdbl', 'pr

In [66]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View


[[(0, 3), (1, 1), (2, 6), (3, 1), (4, 3), (5, 2), (6, 2), (7, 2), (8, 22), (9, 2), (10, 3), (11, 1), (12, 2), (13, 2), (14, 9), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 5), (23, 1), (24, 3), (25, 1), (26, 1), (27, 4), (28, 1), (29, 3), (30, 6), (31, 21), (32, 1), (33, 27), (34, 26), (35, 1), (36, 2), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 2), (47, 2), (48, 2), (49, 2), (50, 1), (51, 1), (52, 11), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 14), (60, 7), (61, 15), (62, 2), (63, 1), (64, 2), (65, 4), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 2), (72, 1), (73, 3), (74, 1), (75, 2), (76, 1), (77, 1), (78, 13), (79, 5), (80, 10), (81, 1), (82, 2), (83, 7), (84, 3), (85, 4), (86, 1), (87, 1), (88, 2), (89, 4), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 2), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 5), (104, 2), (105, 1), (106, 7), (107, 3), (108, 1), (109, 1),

In [67]:
from gensim.models import TfidfModel
tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.05 #TODO: AUTOTUNE HYPERPERAMETERS
#TODO: LDAMULTICORE
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0]not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [72]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)


In [73]:
import pyLDAvis
"https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/"
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=id2word)
vis



/home/liam_mckenna/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/liam_mckenna/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/liam_mckenna/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/liam_mckenna/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/liam_mckenna/.local/lib/python3.8/site-p

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.064222 -0.005618       1        1  18.556850
3      0.010315  0.024872       2        1  15.677339
0     -0.018517  0.049967       3        1  15.567474
4     -0.039839 -0.036922       4        1  15.543926
5      0.007231 -0.040567       5        1  14.687121
2     -0.014895  0.012177       6        1  12.524264
6     -0.008515 -0.003908       7        1   7.443027, topic_info=                Term        Freq       Total Category  logprob  loglift
4990    pleasedshibe  260.000000  260.000000  Default  30.0000  30.0000
2283           video  217.000000  217.000000  Default  29.0000  29.0000
693         nebbiiee  164.000000  164.000000  Default  28.0000  28.0000
26124    thievedcorn   99.000000   99.000000  Default  27.0000  27.0000
18572  liked_youtube   96.000000   96.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
1171           vouch    4.646206   13.341562   Topic7  -7.1983   1.5431
2239          tiktok    3.942044   24.184411   Topic7  -7.3626   0.7839
4215        quackity    3.806975   73.599140   Topic7  -7.3975  -0.3639
837             rank    3.378815   11.405249   Topic7  -7.5168   1.3813
446            grind    3.374246   13.370286   Topic7  -7.5181   1.2210

[333 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
39871      2  0.912995     adambromell
35940      2  0.933738          aheony
7998       4  0.890533         alfiedp
35108      1  0.115850    alicialkrage
35108      7  0.695099    alicialkrage
...      ...       ...             ...
1215       5  0.057457            year
37839      4  0.957911      yuckygrape
44344      5  0.817475        yuumiire
13419      1  0.923821         zapynub
2344       7  0.878777  zetexkindasuck

[452 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 6, 3, 7])

In [75]:
for j in range(7):
    print(j)
    print("-"*40)
    for i, p in lda_model.get_topic_terms(j):
        print(id2word[i], p)

0
----------------------------------------
sure 0.0050961818
thank 0.004776975
back 0.0047312337
stream 0.004691954
fun 0.004361651
time 0.0037708913
consider 0.0036188453
dream 0.0033824425
help 0.0025731775
really 0.0025669306
1
----------------------------------------
stream 0.010735265
live 0.007828649
via_youtube 0.006414722
fxxtrn 0.0048030056
check 0.004142873
nebbiiee 0.003958067
chat 0.003830269
amp 0.0036164026
dream 0.0028368155
astroneergame 0.0024944688
2
----------------------------------------
love 0.005539163
thank 0.0050223833
much 0.0032644568
dream 0.0030580235
ranboo 0.002864315
ranaltboo 0.0028176745
man 0.002111747
ranboosaysstuff 0.0019983598
volterohh 0.0019399724
moot 0.001612604
3
----------------------------------------
dream 0.0052386094
astroneergame 0.0050825696
fuck 0.003990837
say 0.0038582694
stream 0.0033338796
play 0.0029081858
autorasuck 0.002849769
people 0.0024692826
gobills_norcal 0.0022177116
tweet 0.0016205772
4
---------------------------------

In [71]:
lda_model.print_topics()

[(0,
  '0.007*"dream" + 0.005*"sure" + 0.005*"thank" + 0.005*"back" + 0.004*"fun" + 0.004*"stream" + 0.004*"consider" + 0.003*"time" + 0.003*"help" + 0.002*"tomorrow"'),
 (1,
  '0.011*"stream" + 0.008*"live" + 0.007*"via_youtube" + 0.005*"fxxtrn" + 0.004*"check" + 0.004*"chat" + 0.004*"nebbiiee" + 0.004*"amp" + 0.003*"tonight" + 0.003*"astroneergame"'),
 (2,
  '0.003*"ranaltboo" + 0.003*"dream" + 0.002*"gobills_norcal" + 0.002*"man" + 0.002*"ranboo" + 0.002*"volterohh" + 0.002*"ranboosaysstuff" + 0.002*"punz" + 0.001*"taplharv_taplharv" + 0.001*"unu_sually"'),
 (3,
  '0.005*"astroneergame" + 0.004*"dream" + 0.003*"fuck" + 0.003*"autorasuck" + 0.002*"play" + 0.002*"amp" + 0.002*"twitch" + 0.001*"fucking" + 0.001*"quackity" + 0.001*"wooloolove"'),
 (4,
  '0.007*"deytral" + 0.004*"game" + 0.003*"debunkadunk" + 0.003*"follow" + 0.003*"thederpykatlive" + 0.002*"make" + 0.002*"think" + 0.002*"dodoite" + 0.002*"comment" + 0.002*"follower"'),
 (5,
  '0.020*"pleasedshibe" + 0.017*"video" + 0.00